<a href="https://colab.research.google.com/github/farshad36/nlp/blob/main/4P80_Seminar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Joel Gritter & Kindeep Singh Kargil

COSC 4P80 - Seminar Demo

March 29, 2021

In [ ]:
# Music output
!sudo apt-get install fluidsynth
!pip install midi2audio
!pip install mingus
from mingus.containers import Note, NoteContainer, Track
from mingus.midi.midi_file_out import write_NoteContainer, write_Track
from midi2audio import FluidSynth

fsy = FluidSynth()

# imports for data manipulation
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# imports for machine learning
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fluidsynth is already the newest version (1.1.9-1).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.


In [ ]:
# read in the notes, make an array with 0's, except for the current note
def read_and_format(input_filepath):
  input_data = []
  with open(input_filepath) as input_file:
    for line in input_file:
      values = line.split(",")
      for value in values:
        tmp = [0.0] * 88
        v = int(value)
        tmp[v-1] = 1.0
        input_data.append(tmp)
  
  return input_data
  

input_data = read_and_format("k330-allegro-moderato.csv")
print(np.array(input_data).shape)

(2530, 88)


In [ ]:
# get the previous 20 notes, predict the next note
def generate_datasets(input_array, n_prev = 20):
  temp_x = [input_array[i:i+n_prev] for i in range(len(input_array) - n_prev)]
  temp_y = [input_array[i+n_prev] for i in range(len(input_array) - n_prev)]

  return np.array(temp_x), np.array(temp_y)

x, y = generate_datasets(input_data)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, shuffle=False)

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)
print(y_train[0])

(2008, 20, 88) (2008, 88)
(502, 20, 88) (502, 88)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [ ]:
# build the model itself
model = Sequential()
model.add(LSTM(30))
model.add(Dense(88, activation="softmax"))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# train the model
model.fit(x_train, y_train, batch_size=10, epochs=100, validation_split=0.05)

Epoch 1/100
191/191 [==============================] - 5s 13ms/step - loss: 3.7682 - accuracy: 0.1070 - val_loss: 2.4618 - val_accuracy: 0.1683
Epoch 2/100
191/191 [==============================] - 2s 10ms/step - loss: 2.7116 - accuracy: 0.1572 - val_loss: 2.4907 - val_accuracy: 0.2178
Epoch 3/100
191/191 [==============================] - 2s 10ms/step - loss: 2.6787 - accuracy: 0.1633 - val_loss: 2.4384 - val_accuracy: 0.1683
Epoch 4/100
191/191 [==============================] - 2s 10ms/step - loss: 2.6604 - accuracy: 0.1742 - val_loss: 2.3921 - val_accuracy: 0.2178
Epoch 5/100
191/191 [==============================] - 2s 10ms/step - loss: 2.5948 - accuracy: 0.2144 - val_loss: 2.3702 - val_accuracy: 0.2376
Epoch 6/100
191/191 [==============================] - 2s 10ms/step - loss: 2.5757 - accuracy: 0.2170 - val_loss: 2.2860 - val_accuracy: 0.2475
Epoch 7/100
191/191 [==============================] - 2s 10ms/step - loss: 2.4539 - accuracy: 0.2243 - val_loss: 2.1995 - val_accuracy:

In [ ]:
# test the model
model.evaluate(x_test, y_test)

16/16 [==============================] - 0s 4ms/step - loss: 0.7383 - accuracy: 0.7729


[0.7382884621620178, 0.7729083895683289]

In [ ]:
# See incorrectly predicted 
predictions = model.predict(x_test)
incorrect_indices = []
for (index, (prediction, target)) in enumerate(zip(predictions, y_test)):
  pred = np.argmax(prediction)
  tar = np.argmax(target)
  if pred != tar:
    incorrect_indices.append(index)

print(", ".join(map(str, incorrect_indices)))

7, 12, 16, 20, 25, 34, 51, 80, 84, 130, 131, 132, 133, 147, 149, 151, 152, 153, 154, 155, 161, 163, 169, 173, 180, 181, 185, 190, 195, 196, 200, 201, 202, 203, 215, 223, 224, 225, 233, 239, 240, 241, 246, 252, 255, 256, 258, 260, 270, 271, 272, 274, 277, 278, 281, 285, 290, 291, 303, 308, 311, 324, 326, 330, 332, 333, 334, 339, 341, 347, 352, 356, 357, 360, 361, 369, 374, 377, 390, 392, 396, 399, 401, 402, 405, 406, 409, 410, 418, 421, 422, 423, 425, 427, 440, 442, 444, 446, 447, 448, 450, 453, 454, 459, 465, 467, 469, 470, 476, 479, 481, 483, 494, 498


In [ ]:
# initial - provide inital 20 notes
# n - how many predicted notes to add (i.e. expand by this number)
def make_big_song(initial, n):
  res =[ x for x in initial]
  for _ in range(n):
    next = model.predict(np.array([res[-20:],]))[0]
    res.append(next)

  return np.array(res)

# Expects n x 88
def vector_to_midi(arr, filename="nice.midi"):
  track = Track()
  for note_arr in arr:
    note_num = int(np.argmax(note_arr))
    note = Note()
    note.from_int(note_num - 3)
    track.add_notes(note)
  write_Track(filename, track)
  print("Done!")

In [ ]:
full = make_big_song(np.array(input_data[:20]), len(input_data) - 20)

vector_to_midi(full, "full_predicted.midi")

Done!


In [ ]:
def predict_to_file(first_20_notes, expected, filename="nice"):
  next = model.predict(np.array([first_20_notes]))
  actual_next = np.array([expected])
  next_file = filename + "_predicted_note"
  actual_next_file = filename + "_actual_note"
  orig_file = filename + "_first_20_notes"

  vector_to_midi(next, next_file + ".midi")
  vector_to_midi(actual_next, actual_next_file + ".midi")
  vector_to_midi(first_20_notes, orig_file + ".midi")

  # This conversion not seem to work
  # fsy.midi_to_audio(next_file + ".midi", next_file + ".mp3")
  # fsy.midi_to_audio(actual_next_file + ".midi", actual_next_file + ".mp3")
  # fsy.midi_to_audio(orig_file + ".midi",  orig_file + ".mp3")

predict_to_file(test_in, test_out)

inci = incorrect_indices[0]
predict_to_file(x_test[inci], y_test[inci], 'first_incorrect')


Done!
Done!
Done!
Done!
Done!
Done!
